In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

from debug_utils import *

%load_ext autoreload
%autoreload 2

In [2]:
device = get_device()
device

device(type='mps')

In [3]:
batch_size = 64
train_loader, test_loader = load_mnist(batch_size = batch_size)

In [19]:
criterion = nn.CrossEntropyLoss()

In [57]:
images, labels = get_batch(train_loader)
images, labels = images.to(device), labels.to(device)

In [ ]:
leaf_model = CNN().to(device)
model_1 = CNN().to(device)

learning_rate = torch.tensor(1e-3, dtype=torch.float32, requires_grad=True, device=device)

print(learning_rate.grad)

outputs = leaf_model(images)
leaf_model.zero_grad()
loss = criterion(outputs, labels)
loss.backward()

# параметры которе запомнили вычислительный граф
new_params = {name: param - learning_rate * param.grad for name, param in leaf_model.named_parameters()}

# Переносим параметры с одной модели на другую, сохраняя отслеживание градиентов
for (name, param), (_, new_param) in zip(model_1.named_parameters(), new_params.items()):
    param.requires_grad = True  # Сохраняем отслеживание градиентов
    param.data = new_param  

# Прпоускаем градиенты от лосса до весов модели leaf_model и до learning_rate
outputs = model_1(images)
loss = criterion(outputs, labels)
loss.backward()

print(learning_rate.grad)

None
None


tensor(0.0010, device='mps:0', requires_grad=True)

RuntimeError: one of the variables needed for gradient computation has been modified by an inplace operation: [MPSFloatType [10]] is at version 1; expected version 0 instead. Hint: enable anomaly detection to find the operation that failed to compute its gradient, with torch.autograd.set_detect_anomaly(True).

In [102]:
for (name, param), (_, new_param) in zip(model_1.named_parameters(), new_params.items()):
    param.requires_grad = False
    param.copy_(new_param)
    print(param.is_leaf)

for (name, param), (_, new_param) in zip(model_1.named_parameters(), new_params.items()):
    print((param == new_param), param.is_leaf)



False
False
False
False
False
False
False
False
tensor([[[[True, True, True],
          [True, True, True],
          [True, True, True]]],


        [[[True, True, True],
          [True, True, True],
          [True, True, True]]],


        [[[True, True, True],
          [True, True, True],
          [True, True, True]]],


        [[[True, True, True],
          [True, True, True],
          [True, True, True]]],


        [[[True, True, True],
          [True, True, True],
          [True, True, True]]],


        [[[True, True, True],
          [True, True, True],
          [True, True, True]]],


        [[[True, True, True],
          [True, True, True],
          [True, True, True]]],


        [[[True, True, True],
          [True, True, True],
          [True, True, True]]],


        [[[True, True, True],
          [True, True, True],
          [True, True, True]]],


        [[[True, True, True],
          [True, True, True],
          [True, True, True]]],


        [[[T

In [88]:
for (name, param), (_, new_param) in zip(model_1.named_parameters(), new_params.items()):
    # print(param == new_param)
    print(param.is_leaf , new_param.is_leaf)

True False
True False
True False
True False
True False
True False
True False
True False


In [84]:
learning_rate.grad

In [77]:
for name, param in leaf_model.named_parameters():
    param.grad.zero_()
    
    print(name , param.grad is not None)
    new_param = param * learning_rate
    print(param.is_leaf , learning_rate.is_leaf, new_param.is_leaf)

    value = torch.sum(new_param**2)
    value.backward()

    print(param.grad )

conv1.weight True
True True False
tensor([[[[ 3.9311e-07,  6.5936e-07,  1.9188e-07],
          [ 1.9255e-07, -5.9283e-07, -5.3736e-07],
          [-5.4589e-07, -1.1215e-08, -5.9124e-07]]],


        [[[-4.0781e-07,  1.9696e-07,  3.6641e-07],
          [-2.6646e-07, -6.4334e-07,  2.6286e-07],
          [-6.1491e-07,  4.5499e-07,  5.3554e-07]]],


        [[[-5.4482e-07, -3.0979e-07,  5.9717e-07],
          [ 6.3735e-07, -6.5756e-08,  7.2743e-08],
          [ 4.5395e-08,  1.7619e-07,  3.6919e-07]]],


        [[[-9.7291e-08,  5.0710e-08, -7.9009e-08],
          [-3.6320e-08,  5.1337e-07,  7.2386e-08],
          [-4.2262e-07, -1.9121e-07,  2.0234e-08]]],


        [[[ 2.7434e-07,  2.7896e-08, -4.5730e-07],
          [ 6.5972e-07, -3.8540e-07,  2.4600e-07],
          [ 6.3775e-07, -2.8126e-07, -1.9622e-07]]],


        [[[ 5.6632e-07, -2.0528e-08,  2.3713e-07],
          [-4.1091e-07,  6.5803e-07,  3.8207e-07],
          [-5.1451e-07, -3.6457e-07, -2.1865e-08]]],


        [[[-1.3948e-07, 

False
False
False
False
False
False
False
False


In [80]:
outputs = model_1(images)
loss = criterion(outputs, labels)
loss.backward()

In [81]:
learning_rate.grad

tensor(0.9463, device='mps:0')